In [249]:
import sys
sys.path.append("/richmondvol1/rusty/stompy")

In [250]:
from stompy import utils
import six
import shutil
import stompy.model.delft.dflow_model as dfm
import stompy.model.delft.waq_scenario as dwaq
import subprocess
import os

In [251]:
if 1:
    dfm_path="/chicagovol1/hpcshared/open_bay/hydro/full_res/wy2022_bloom/runs/wy2022_bloom_with_temp"
    dwaq_hydro=os.path.join(dfm_path, "DFM_DELWAQ_wy2022_bloom_with_temp")
    hydro=dwaq.HydroFiles(os.path.join(dwaq_hydro,"wy2022_bloom_with_temp.hyd"))
if 0:
    # non-temperature run gets us 2 more days of output. actually no. valid data stops
    # 8/21 23:00,while the temperature runs goes to 8/23 00:00
    dfm_path="/chicagovol1/hpcshared/open_bay/hydro/full_res/wy2022_bloom/runs/wy2022_bloom"
    dwaq_hydro=os.path.join(dfm_path, "DFM_DELWAQ_wy2022_bloom")
    hydro=dwaq.HydroFiles(os.path.join(dwaq_hydro,"wy2022_bloom.hyd"))

In [252]:
# wholesale copypasta https://github.com/rustychris/agg_wy2013/blob/master/tracer_tests/tracer_common.py
import logging
log = logging.getLogger()

import re
import matplotlib.pyplot as plt
from matplotlib import colors
from shapely import geometry, wkt
import stompy.plot.cmap as scmap
import xarray as xr
from stompy.grid import unstructured_grid
import numpy as np
import pandas as pd
import stompy.model.delft.waq_scenario as dwaq
from stompy.memoize import memoize
import logging as log
from stompy import utils
from stompy import filters
from stompy.plot import plot_wkb
import netCDF4
import warnings
from stompy.model.data_comparison import calc_metrics
import os
import six
from matplotlib import gridspec

cmap=scmap.load_gradient('turbo.cpt') # a less offensive 'jet'

In [253]:
# Regions (copy paste from QGIS vector layer)
oakland_wkt="""Polygon ((560434.67213132162578404 4183884.9798020776361227, 
567612.90416294464375824 4183342.8737371894530952, 571557.19311782089062035 
4177809.65321281319484115, 570472.98098804452456534 4175697.30889100721105933, 
569052.2892317856894806 4173491.49800628982484341, 567612.90416294464375824 
4172893.3120036544278264, 565014.53371399780735373 4172930.69862881954759359, 
562920.88270477438345551 4174201.84388441918417811, 562528.32314054504968226 
4175510.37576518394052982, 562976.96264252148102969 4181174.44947763625532389,
560042.11256709217559546 4182651.2211716421879828, 560434.67213132162578404 
4183884.9798020776361227))
"""
oakland_poly=wkt.loads(oakland_wkt)
# oakland_poly

In [254]:
eastshore_wkt="""Polygon ((568776.56287119572516531 4174108.37732150498777628,
570122.4813771250192076 4175678.61557842278853059, 573487.2776419484289363
4172538.13906458765268326, 575842.63502732466440648 4169790.22211498161777854, 
575898.71496507176198065 4167042.30516537604853511, 577188.55353325395844877 
4160312.71263572946190834, 577917.59272396576125175 4160144.47282248828560114, 
582460.06768147717230022 4158125.59506359416991472, 580777.66954906552564353 
4156667.51668217079713941, 577188.55353325395844877 4157172.23612189432606101, 
576627.75415578344836831 4156555.3568066768348217, 575618.31527633650694042 
4157789.11543711181730032, 574664.95633463654667139 4160200.55276023549959064, 
574160.23689491301774979 4163060.62958533503115177, 573767.67733068368397653 
4165920.7064104350283742, 572814.3183889837237075 4169509.82242624647915363, 
570683.28075459564570338 4171416.54030964663252234, 568776.56287119572516531 
4174108.37732150498777628))
"""
eastshore_poly=wkt.loads(eastshore_wkt)
# eastshore_poly

In [279]:
def hydro_name(hydro): return "wy2022_bloom"

def configure_dwaq(): # May be outdated, esp. for chicago.
    # configure DWAQ:
    DELFT_SRC="/opt/software/delft/delwaq/precompiled_binaries/DFM1.6.2.49199/lnx64"
    #DELFT_SRC="/home/alliek/software/Delft3D-FM/64634"
    DELFT_SHARE=os.path.join(DELFT_SRC,"share","delft3d")
    DELFT_LIB=os.path.join(DELFT_SRC,"lib")

    os.environ['DELFT_SRC']=DELFT_SRC
    os.environ['DELFT_SHARE']=DELFT_SHARE
    
    if 'LD_LIBRARY_PATH' in os.environ:
        os.environ['LD_LIBRARY_PATH']=DELFT_LIB+":"+os.environ['LD_LIBRARY_PATH']
    else:
        os.environ['LD_LIBRARY_PATH']=DELFT_LIB

configure_dwaq()
dfm_bin=os.path.join(os.environ['DELFT_SRC'],'bin')
waqpbexport=os.path.join(dfm_bin,'waqpbexport')
waqpbimport=os.path.join(dfm_bin,'waqpbimport')

class CommonSetup(object):
    """
    Common code for various tracers runs
    """
    name='common' # should overload
    hydro=None
    base_path=None # must be set!

    force=True # whether to allow re-using an existing run
    
    # start time offset from start of hydro by this delta
    # give it some decent spinup time
    start_time=np.datetime64("2022-08-10 00:00")
    
    # set length of the run. Appears to be the end of valid output, even though the
    # hydro reports going until the 25th.
    stop_time=np.datetime64("2022-08-22 23:00") # seems that even temperature run is dicey on last step
    
    integration_option="15.60" # if set, copied to WaqModel
    time_step=3000
    map_time_step=3000 # otherwise it will default to time_step, which could be really short.    

    waq_kws={}
    def __init__(self,**kw):
        utils.set_keywords(self,kw)   
        if self.base_path is None:
            self.base_path=self.calc_base_path()
                
            yyyymmdd=utils.to_datetime(self.start_time).strftime('%Y%m%d')
            self.base_path+="_%s"%(yyyymmdd)   
            
            # And make it unique on successive runs
            for seq in range(50):
                test_path=self.base_path
                if seq>0:
                    test_path+=f"-v{seq:03}"
                if not os.path.exists(test_path):
                    self.base_path=test_path
                    break
            else:
                raise Exception("Too many runs with same name")
            
            log.info("base_path defaults to %s"%self.base_path)
    def calc_base_path(self):
        p='run_%s_%s'%(hydro_name(self.hydro),self.name)
        return p
        
    def release_conc_3d(self,*a,**kw):
        C_2d=self.release_conc_2d(*a,**kw)
        C_3d=self.hydro.extrude_element_to_segment(C_2d)
        return C_3d

    def setup_model(self):
        # Create a WaqModel, add some tracers
        self.wm=wm=dwaq.WaqModel(hydro=self.hydro,
                                 overwrite=True,
                                 base_path=self.base_path,
                                 mon_time_step=1000000, # daily
                                 time_step=self.time_step,
                                 **self.waq_kws)
        # add some option for balances.
        wm.integration_option="%s BALANCES-OLD-STYLE BAL_NOLUMPPROCESSES BAL_NOLUMPLOADS BAL_NOLUMPTRANSPORT"%self.integration_option
        #wm.start_time+= self.start_offset
        wm.start_time = self.start_time # may have to be smarter about starting on an output time step.
        # hydro reports the wrong stop time. manually set.
        if self.stop_time is not None:
            wm.stop_time=self.stop_time
        
        self.setup_tracers()
        
        wm.parameters['ACTIVE_VertDisp']=1
        wm.parameters['ScaleVDisp']=1.0 
        
    def run_waq_model(self):
        assert self.base_path is not None,"Must specify base_path"
        
        if not self.force:
            if os.path.exists(os.path.join(self.base_path,'dwaq_map.nc')):
                log.info("Run seems to exist -- will not run again")
                self.wm=dwaq.WaqModel.load(self.base_path,load_hydro=False)
                return

        self.setup_model()
        
        wm=self.wm
        wm.cmd_write_hydro()
        wm.cmd_write_inp()
        self.copy_notebook()        
        wm.cmd_delwaq1()
        wm.cmd_delwaq2()
        wm.cmd_write_nc()
    def copy_notebook(self):
        script_fn="run_tracers_age.ipynb"
        shutil.copyfile(script_fn,os.path.join(self.base_path,script_fn))
    def setup_tracer_continuity(self):
        # continuity tracer:
        self.wm.substances['continuity']=dwaq.Substance(initial=1.0)
        # This adds a concentration=1.0 boundary condition on all the boundaries.
        all_bcs=[b.decode() for b in np.unique(self.hydro.boundary_defs()['type'])]
        self.wm.add_bc(all_bcs,'continuity',1.0)
    
        
class Blobber(CommonSetup):
    # plume parameters
    X0=np.array([564560,4.17838e6])
    L=2000
    fig_num=None
    zoom=(552164., 595434., 4138443.,4186236.)
    txt_loc=[0.05,0.15] # axes coordinate for location of label
    name='gaussian' # used to form default base_path
    
    def release_conc_2d(self,X0,L):
        grid=self.hydro.grid()
        X=grid.cells_centroid()
        
        c=np.exp( -((X-X0)**2).sum(axis=-1)/L**2 )
        c=c/c.max() # make max value 1
        return c

    def setup_tracers(self):
        # boundary condition will default to 0.0
        C_3d=self.release_conc_3d(X0=self.X0,L=self.L)
        self.wm.substances['dye1']=dwaq.Substance(initial=C_3d)
        self.setup_tracer_continuity()
    
class PolyBlobber(CommonSetup):
    """
    Seed with regional dye releases
    """
    # plume parameters
    release_polys=[(oakland_poly,'oakland'),
                   #(eastshore_poly,'eastshore')
                  ]
    
    def release_conc_2d(self,poly):
        grid=self.hydro.grid()
        # This could also be used to sample from satellite data.
        c=grid.select_cells_intersecting(poly,by_center='centroid')
        c=c.astype(np.float64)
        return c
        
    def setup_tracers(self):
        for poly,name in self.release_polys:
            # boundary condition will default to 0.0
            C_3d=self.release_conc_3d(poly=poly)
            self.wm.substances[name]=dwaq.Substance(initial=C_3d)
                    
    # Plotting is old -- not yet updated for this application.
    def figure_2panels_and_time(self):
        ds=xr.open_dataset(os.path.join(self.wm.base_path,'dwaq_map.nc'))
        his_ds=xr.open_dataset(os.path.join(self.wm.base_path,'dwaq_hist.nc'))
        grid_ds=unstructured_grid.UnstructuredGrid.from_ugrid(ds)
        
        fig=plt.figure(self.fig_num)
        gs=gridspec.GridSpec(1,4)
        
        axs=[fig.add_subplot(gs[:,0]),
             fig.add_subplot(gs[:,1])]
        ax_t=fig.add_subplot(gs[:,2:])
        self.fig=fig
        fig.set_size_inches([12,5],forward=True)
        
        scal='dye1'
        # initial
        ax=axs[0]
        ax.text(self.txt_loc[0],self.txt_loc[1],"Initial",transform=ax.transAxes,va='top')
        coll_ic=grid_ds.plot_cells(values=ds[scal].isel(time=0,layer=0),ax=ax,cmap=self.cmap,
                                   clim=[0,1])
        # at the end
        ti=len(ds.time)-1
        ax=axs[1]
        ax.text(self.txt_loc[0],self.txt_loc[1],"Final",transform=ax.transAxes,va='top')
        coll_ec=grid_ds.plot_cells(values=ds[scal].isel(time=-1,layer=0),ax=ax,cmap=self.cmap,
                                   clim=[0,1])
        
        for coll in [coll_ic,coll_ec]:
            coll.set_lw(0.75)
            coll.set_edgecolor('face')
            
        for ax in axs[:2]:
            ax.xaxis.set_visible(0)
            ax.yaxis.set_visible(0)
            ax.axis('equal')
            ax.axis(self.zoom)
            plt.colorbar(ax.collections[0],orientation='horizontal',ax=ax,fraction=0.08,pad=0.05)
            
        ax_t.plot(his_ds.time,his_ds.bal.sel(region='region',field='dye1'))
            
        fig_title=self.hydro.hyd_path
                    
        fig.text(0.5,0.95,fig_title,ha='center')
        fig.tight_layout()
        fig.subplots_adjust(top=0.93) # make some space for fig text
                    
        ds.close() # keeping this open can interfere with deleting or overwriting the netcdf file.
        his_ds.close()
        
class SurfaceBlobber(PolyBlobber):
    dz_surface=2.0
    def release_conc_3d(self,*a,**kw):
        C_2d=self.release_conc_2d(*a,**kw)
        C_3d=self.hydro.extrude_element_to_segment(C_2d)

        # And now limit to the surface
        start_t_secs=int( (self.start_time - utils.to_dt64(self.hydro.time0))/np.timedelta64(1,'s'))
        seg_ztop,seg_zbot = self.hydro.seg_z_range(start_t_secs)

        dz=seg_zbot-seg_ztop
        dz_release=(seg_zbot.clip(None,self.dz_surface) - seg_ztop).clip(0,None)
        frac=dz_release/dz
        C_3d *= frac

        return C_3d
    

def recalc_age(cons,decay,rate=0.01,min_cons=1e-7):
    # There are some numerical issues with the age tracers, notably
    # that in some cases the decaying tracer is greater than the
    # conservative tracer.
    # recalculate age to have a better sense of where
    # the nan's are.
    # the rate here is the default in dwaq, with units of day-1
    
    # possible for cons or decay to be nan, 0, or for cons<decay,
    # all of which are problematic.
    with np.errstate(invalid='ignore',divide='ignore'):
        valid=(cons>1e-7)
        my_age=np.log(decay/cons) / np.log(1-rate)
        my_age[ cons<min_cons]=np.nan
        my_age[ decay>cons]=0.0
        my_age[np.isnan(cons*decay)]=np.nan 
    return my_age



In [280]:
if 0: # moved on to continuous source cases, but might return to instant, region release.
    pb=PolyBlobber(hydro=hydro)
    #pb=SurfaceBlobber(hydro=hydro)
    try:
        pb.run_waq_model()
    except:
        # If the run doesn't fail out, this isn't necessary
        pb.wm.cmd_write_nc()
        raise

Custom processes
--

This is the low-level approach. There may be some Delft tools to automate this, but
my general experience is that the tools get out of date and become unsupported.

Each process needs two pieces:
 1. Fortran code that implements the operation
 2. Tables that associate a process name, inputs, and outputs with the fortran function.
 
The same fortran code is reused for similar processes. The tables can define multiple
entries that use the same fortran implementation.

This means there is the "simple" level of customizing processes (edit the tables), and
the "complete" level (write new fortran code, along with tables to use the new fortran
subroutine).

Compiling new fortran subroutines requires using a compiler similar to what was used to
compile the original delwaq library. I think Deltares tend to use intel compilers, and
intel compilers are generally compatible with gnu compilers, so this isn't necessarily
a major hurdle (and intel compilers are readily available for free now). Nonetheless,
getting everything to work with a bespoke fortran subroutine is more error-prone than
just editing the tables.

The tables are stored three different ways, which makes the editing process at first
confusing.
 - proc_def.def and proc_def.dat: binary (NEFIS), read by delwaq at runtime
 - proces.asc: the human (for some value of human) readable table
 - a bunch of CSV files
 
You'd want to just edit proces.asc, and generate proc_def.*, but no. You have to edit
proces.asc, convert that to CSV (`waqpbimport`), and then convert back (`waqpbexport`).
Also, even though `waqpbimport` updates the CSV files, it requires them to already
exist.

You'd expect that binary distribution of delwaq would come with the CSVs, but no.


So this process is tricky with a binary compile, even if you don't want to compile.


In [291]:
# path to clean set of csv files, though potentially wrong DWAQ version
table_src="waq_tables_rusty_laptop"
# path to where edits, import/export will happen
table_dst="waq_tables_local"

# Clean slate:
os.path.exists(table_dst) and shutil.rmtree(table_dst)
shutil.copytree(table_src,table_dst)

print( utils.call_with_path(waqpbexport,table_dst).decode('latin1') )
# => proc_def.* and procesm.asc in table_dst

# First line of proces_asc includes a process count. Rewrite that line, shove in
# our new process, and copy the rest of the file.

# Names must be <=10 characters!

def CART(name="CART1",
         zero_order="ZAge1",
         partial="PartAge1",
         conc="Age1Conc",
         age_conc="Age1AConc",
         flux="dAge1",
         conc_decay=0.0):
    n_stoich=2
    if conc_decay!=0.0:
        #conc_stoich=f"ALKA1       {flux:10}    -8.71400"
        n_stoich+=1
    process=f"""
{name}                    Reuse nitrification as age                         
NITRIF    ; module name. 
123       ; TRswitch
        18; # input items for segments
{zero_order:10}      0.00000     x zeroth-order nitrification flux          (gN/m3/d)
{partial   :10}      1.00000     x set this to get partial age
RcNit20        0.100000     x ignored (b/c SWVnNit=0)
TcNit           1.00000     x ignored
OXY             10.0000     x ignored
KsAmNit        0.500000     x ignored
KsOxNit         1.00000     x ignored
Temp            15.0000     x ignored
CTNit           3.00000     x ignored
Rc0NitOx        0.00000     x ignored
COXNIT          1.00000     x ignored
Poros           1.00000     x volumetric porosity                            (-)
SWVnNit         0.00000     x switch for old (0), new (1), TEWOR (2) version (-)
{conc    :10}     0.100000     x concentration tracer
OOXNIT          5.00000     x ignored
CFLNIT          0.00000     x ignored
CurvNit         0.00000     x ignored
DELT           -999.000     x timestep for processes                         (d)
         0; # input items for exchanges
         1; # output items for segments
O2FuncNT1                   x oxygen function for nitrification              (-)
         0; # output items for exchanges
         1; # fluxes
{flux:10}                  x nitrification flux                       (gN/m3/d)
         {n_stoich}; # stoichiometry lines. Could probably drop most of these.
NH4         {flux:10}    -1.00000
{age_conc:10}  {flux:10}     1.00000
"""
    if conc_decay!=0.0:
        process+=f"{conc:10}  {flux:10}    {-conc_decay:.5f}\n"
    process+="""         0; # stoichiometry lines dispersion arrays
         0; # stoichiometry lines velocity arrays
END
"""
    return process

# Need some way to keep the discharger tracers from just accumulating
# ad hoc, set a 0.1/day decay rate.
# Would like tracer age while we're at it, so just create a matching pair
# of conservative discharge tracers.
discharge_decay_rate=0.1

new_procs=[
    CART(name="CART1",zero_order="ZAge1",partial="PartAge1",conc="Age1Conc",
         age_conc="Age1AConc",flux="dAge1"),
    CART(name="CART1D",zero_order="ZAge1",partial="LocalDepth",conc="Age1Conc",
         age_conc="Age1DAConc",flux="dAgeD1"),
    CART(name="CART1N",zero_order="ZAge1",partial="EBDA",conc="Age1Conc",
         age_conc="Age1NAConc",flux="dAgeN1"),
    CART(name="CART1L",zero_order="ZAge1",partial="LSB",conc="Age1Conc",
         age_conc="Age1LAConc",flux="dAgeL1"),
    # and a second-moment-concentration tracer (v for variance)
    # first try I had partial=Age1AConc, conc=Age1Conc. But I think that
    # ends up squaring concentration.
    CART(name="CART1V",zero_order="ZAge1",partial="PartAge1",conc="Age1AConc",
         age_conc="Age1VConc",flux="dAgeV1"),
    # And a 0.1/day decay for the sources
    CART(name="NDECAY",zero_order="ZAge1",partial="PartAge2",conc="EBDA",
         age_conc="Age2AConc",flux="dAge2",conc_decay=discharge_decay_rate),
    CART(name="LDECAY",zero_order="ZAge1",partial="PartAge2",conc="LSB",
         age_conc="Age3AConc",flux="dAge3",conc_decay=discharge_decay_rate),
    # Originally used partial='Rad', but then all surface layers have the same
    # value, and we lose the depth dependence.
    CART(name="CART1Rad",zero_order="ZAge1",partial="RadBot",conc="Age1Conc",
        age_conc="Age1RadAge",flux='dAgeR1'),
]

procesm_asc=os.path.join(table_dst,'procesm.asc')
proces_asc=os.path.join(table_dst,'proces.asc')

# funny characters in the incoming file.
if 1:
    with open(procesm_asc,'rt',encoding='latin1') as fp_in:
        with open(proces_asc,'wt') as fp:
            hdr=fp_in.readline().split()
            count=int(hdr[0]) + len(new_procs)        
            fp.write(f"{count:10} {hdr[1]:>57} {hdr[2]:>11}\n")
            for proc in new_procs:
                fp.write(proc.strip()+"\n")
            for line in fp_in:
                fp.write(line)
else:
    shutil.copy(procesm_asc,proces_asc)

print( utils.call_with_path(waqpbimport,table_dst).decode('latin1') )
print( utils.call_with_path(waqpbexport,table_dst).decode('latin1') )

Normal end


 Reading data......
 Writing TRM tables for LaTeX......
 Making PROCES.ASC......
 
+Process: pH_carb   
+Process: DynDepth  
+Process: DynSurf   
+Process: TotDepth  
+Process: Emersion  
+Process: Meteo     
+Process: HeatBal   
+Process: HeatBal2  
+Process: SelfCool  
+Process: MaxMacro  
+Process: Coverage  
+Process: DAYRAD    
+Process: TempMode  
+Process: Temperatur
+Process: VarSal    
+Process: Salinchlor
+Process: Veloc     
+Process: ResTim    
+Process: VertDisp  
+Process: HorzDisper
+Process: HDisperVel
+Process: HDisperAdd
+Process: Age1      
+Process: Age2      
+Process: Age3      
+Process: Age4      
+Process: Age5      
+Process: WFetch    
+Process: WDepth    
+Process: Chezy     
+Process: Wave      
+Process: CalTau    
+Process: MACDIS01  
+Process: MACDIS02  
+Process: MACDIS03  
+Process: MACDIS04  
+Process: MACDIS05  
+Process: pH_simp   
+Process: SpecCarb  
+Process: EXTINABVLP
+Process: EXTINABVL 
+Process: ExtPhDVL  
+Process: EXTINABUVP
+Process: EXTI

Normal end


 Loading database......
 
 Decomposing proces.asc                                                                      ......
 
Process: CART1     
Process: CART1D    
Process: CART1N    
Process: CART1L    
Process: CART1V    
Process: NDECAY    
Process: LDECAY    
Process: CART1Rad  
Process: pH_carb   
Process: DynDepth  
Process: DynSurf   
Process: TotDepth  
Process: Emersion  
Process: Meteo     
Process: HeatBal   
Process: HeatBal2  
Process: SelfCool  
Process: MaxMacro  
Process: Coverage  
Process: DAYRAD    
Process: TempMode  
Process: Temperatur
Process: VarSal    
Process: Salinchlor
Process: Veloc     
Process: ResTim    
Process: VertDisp  
Process: HorzDisper
Process: HDisperVel
Process: HDisperAdd
Process: Age1      
Process: Age2      
Process: Age3      
Process: Age4      
Process: Age5      
Process: WFetch    
Process: WDepth    
Process: Chezy     
Process: Wave      
Process: CalTau    
Process: MACDIS01  
Process: MACDIS02  
Process: MACDIS03  
Process: MAC

 Reading data......
 Writing TRM tables for LaTeX......
 Making PROCES.ASC......
 
+Process: CART1     
+Process: CART1D    
+Process: CART1N    
+Process: CART1L    
+Process: CART1V    
+Process: NDECAY    
+Process: LDECAY    
+Process: CART1Rad  
+Process: pH_carb   
+Process: DynDepth  
+Process: DynSurf   
+Process: TotDepth  
+Process: Emersion  
+Process: Meteo     
+Process: HeatBal   
+Process: HeatBal2  
+Process: SelfCool  
+Process: MaxMacro  
+Process: Coverage  
+Process: DAYRAD    
+Process: TempMode  
+Process: Temperatur
+Process: VarSal    
+Process: Salinchlor
+Process: Veloc     
+Process: ResTim    
+Process: VertDisp  
+Process: HorzDisper
+Process: HDisperVel
+Process: HDisperAdd
+Process: Age1      
+Process: Age2      
+Process: Age3      
+Process: Age4      
+Process: Age5      
+Process: WFetch    
+Process: WDepth    
+Process: Chezy     
+Process: Wave      
+Process: CalTau    
+Process: MACDIS01  
+Process: MACDIS02  
+Process: MACDIS03  
+Process: MACD

Normal end


In [287]:
class LocalPolyBlobber(PolyBlobber):
    def setup_model(self):
        super().setup_model()
        self.wm.proc_path=os.path.abspath(os.path.join(table_dst,'proc_def'))
        
    def setup_tracers(self):
        for poly,name in self.release_polys:
            # boundary condition will default to 0.0
            C_3d=self.release_conc_3d(poly=poly)
            self.wm.substances[name]=dwaq.Substance(initial=C_3d)
        # setup an age tracer for the first one.
        poly,name=self.release_polys[0]
        C_3d=self.release_conc_3d(poly=poly)
        self.wm.substances['Age1Conc']=dwaq.Substance(initial=C_3d)
        self.wm.substances['Age1AConc']=dwaq.Substance(initial=0.0)
        self.wm.substances['Age1DAConc']=dwaq.Substance(initial=0.0)
        self.wm.add_process(name="CART1")
        self.wm.add_process(name="CART1D")

if 0:
    # short run for testing.
    # Check argument to delwaq1
    # Original: /opt/software/delft/delwaq/precompiled_binaries/DFM1.6.2.49199/lnx64/bin/delwaq1 -waq  -p /opt/software/delft/delwaq/precompiled_binaries/DFM1.6.2.49199/lnx64/share/delft3d/proc_def

    # Choose a time early in the hydro to decrease seek time (?)
    pb=LocalPolyBlobber(hydro=hydro)
    #                    start_time=np.datetime64("2022-05-15 00:00"),
    #                    stop_time =np.datetime64("2022-05-15 02:00"))

    pb.run_waq_model()

In [297]:
class LocalSpigot(CommonSetup):
    x0=[564750, 4182370]
    # start earlier since we're tracking age now.
    # but that's inexact. start at the time of release
    start_time=np.datetime64("2022-08-02 00:00")
    # DEV
    #start_time=np.datetime64("2022-08-04 00:00")
    #stop_time=np.datetime64("2022-08-08 00:00") 
    
    kd_path="../Kd_2022/Kd_sentinel3_1h/Kd_sent3_20220801_20220901.nc"
    def setup_model(self):
        super().setup_model()
        self.wm.proc_path=os.path.abspath(os.path.join(table_dst,'proc_def'))
        
    def setup_tracers(self):
        self.add_light()

        #self.add_ebda()
        #self.add_lsb()
        
        self.wm.substances['Age1Conc']=dwaq.Substance(initial=0.0)
        # add source as a discharge
        elt=self.hydro.grid().select_cells_nearest(self.x0) # get the 2D cell
        discharge=self.wm.add_discharge(element=elt,k=0) # dump into surface layer of it
        self.wm.add_load([discharge],['Age1Conc'],10000.0) # stave off some 32-bit roundoff

        # age tracer
        self.wm.add_process(name="CART1") 
        self.wm.substances['Age1AConc']=dwaq.Substance(initial=0.0)
        
        # depth exposure
        self.wm.substances['Age1DAConc']=dwaq.Substance(initial=0.0)
        self.wm.add_process(name="CART1D")
        
        # age variance
        self.wm.substances['Age1VConc']=dwaq.Substance(initial=0.0)        
        self.wm.add_process(name="CART1V")        
        
        # Light exposure tracer
        self.wm.substances['Age1RadAge']=dwaq.Substance(initial=0)
        self.wm.add_process("CART1Rad")
        
    def add_ebda(self):
        self.wm.substances['EBDA']=dwaq.Substance(initial=0.0)
        self.wm.substances['EBDAcon']=dwaq.Substance(initial=0.0)
        # exposure to EBDA discharge
        self.wm.substances['Age1NAConc']=dwaq.Substance(initial=0.0)
        self.wm.add_process(name="CART1N")
        # hardwired 0.1/day decay for EBDA and LSB
        self.wm.add_process(name="NDECAY")
        # mock EBDA
        x_EBDA=[562257.0,4172050.8] # picked off a QGIS map
        elt=self.hydro.grid().select_cells_nearest(x_EBDA) # get the 2D cell
        discharge=self.wm.add_discharge(element=elt,k=5) # dump mid water column
        self.wm.add_load([discharge],['EBDA','EBDAcon'],10000.0) # stave off some 32-bit roundoff
        
    def add_lsb(self):
        self.wm.substances['LSB']=dwaq.Substance(initial=0.0)
        self.wm.substances['LSBcon']=dwaq.Substance(initial=0.0)
        # exposure to LSB discharge
        self.wm.substances['Age1LAConc']=dwaq.Substance(initial=0.0)            
        self.wm.add_process(name="CART1L")            
        self.wm.add_process(name="LDECAY")

        ## Lump LSB sources -- one point in Coyote Ck
        x_LSB=[585597,4146720]
        elt=self.hydro.grid().select_cells_nearest(x_LSB) # get the 2D cell
        discharge=self.wm.add_discharge(element=elt,k=5) # dump mid water column
        self.wm.add_load([discharge],['LSB','LSBcon'],10000.0) # stave off some 32-bit roundoff

    def add_light(self):
        if self.kd_path is None: return
        self.add_kd()
        # For the moment stick to constant, daily-averaged insolation, but use the 
        # regular calc rad
        self.wm.add_process(name='CalcRad')
        self.add_insolation()
        
        self.wm.map_output += ('RadBot','ExtVl')
    def add_insolation(self):
        if 0:
            self.wm.parameters['RadSurf']=500.0 
        else:
            cimis=xr.open_dataset('/richmondvol1/hpcshared/inputs/cimis/union_city-hourly-2022_bloom.nc')
            # Starts as PST, but the model is UTC.
            cimis=cimis.set_coords('time').swap_dims({'Date':'time'})
            cimis['time']=cimis['time']+np.timedelta64(8,'h')
            sol_rad=cimis['HlySolRad'].values
            sol_rad=utils.fill_invalid(sol_rad)
            
            t0=np.datetime64(self.hydro.time0)
            t_secs=((cimis.time.values-t0)/np.timedelta64(1,'s')).astype(np.int64)
            param=dwaq.ParameterTemporal(times=t_secs,values=sol_rad)
            self.wm.parameters['RadSurf'] = param
            
    def add_kd(self):
        # extrude to 3D, write seg function
        ds=xr.open_dataset(self.kd_path)
        g=unstructured_grid.UnstructuredGrid.read_ugrid(ds)
        assert np.allclose(g.cells_centroid(), self.hydro.grid().cells_centroid()),"Kd field grid does not match"

        t0=np.datetime64(self.hydro.time0)
        tsecs=(ds.time.values-t0)/np.timedelta64(1,'s')
        def seg_func(t):
            C_2d=ds['Kd'].sel(time=t0+t*np.timedelta64(1,'s'),method='nearest').values
            return self.hydro.extrude_element_to_segment(C_2d)
            
        # No self-shading, specify overall extinction directly
        self.wm.parameters['ExtVl'] = dwaq.ParameterSpatioTemporal(times=tsecs,func_t=seg_func)
        

spigot=LocalSpigot(hydro=hydro)
spigot.run_waq_model()

INFO:root:base_path defaults to run_wy2022_bloom_common_20220802-v003
INFO:WaqModel: start time updated from hydro: 2022-05-01T00:00:00.000000
INFO:WaqModel: stop time update from hydro: 2022-08-23T00:00:00.000000
INFO:HydroFiles:Segment depth will be inferred
INFO:WaqModel:Parameters gleaned from hydro: NamedObjects([('surf', <stompy.model.delft.waq_scenario.ParameterSpatial object at 0x7f44300ce7c0>), ('bottomdept', <stompy.model.delft.waq_scenario.ParameterSpatial object at 0x7f44300ce6d0>), ('vertdisper', <stompy.model.delft.waq_scenario.ParameterSpatioTemporal object at 0x7f44300ce760>), ('tau', <stompy.model.delft.waq_scenario.ParameterSpatioTemporal object at 0x7f44300ce880>), ('temp', <stompy.model.delft.waq_scenario.ParameterSpatioTemporal object at 0x7f44300ce700>), ('salinity', <stompy.model.delft.waq_scenario.ParameterSpatioTemporal object at 0x7f44300ce8e0>)])
INFO:WaqModel:Writing hydro data
INFO:HydroFiles:Using .bnd file, not writing out kludgey boundary-links.csv
INFO: